# Healthym Chat Agent
The goal of this notebook is to build a chat agent for Healthym, a fictional company in the healthy foods industry. To build the chatbot, a large language model will be used alongisde Retrieval Augmented Generation and the company's knowledge base.

In [1]:
# Relevant imports

import os
import glob
from dotenv import load_dotenv
import numpy as np
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
import gradio as gr

In [2]:
# Load environment variables

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [3]:
# Define model to be used and a database name

MODEL = "gpt-4o-mini" # This can be changed to another model. This one was set due to its high cost efficiency
db_name = "vector_db"

In [17]:
folders = glob.glob("knowledge_base/*")
folders

['knowledge_base\\company',
 'knowledge_base\\products',
 'knowledge_base\\recipes',
 'knowledge_base\\suppliers']

In [18]:

# Set paths for folders in the company's knowledge base

folders = glob.glob("knowledge_base/*")

# Set encoding for reading the text documents
text_loader_kwargs = {'encoding': 'utf-8'}

# Load documents using LagChain loaders
documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

In [19]:
# Sample document

documents[0]

Document(metadata={'source': 'knowledge_base\\company\\company_summary.md', 'doc_type': 'company'}, page_content="==============================================\n\nHealthym: Nourishing Communities with Healthy Foods\n=====================================================\n\n### Summary\n\nHealthym is a pioneering company dedicated to providing high-quality, healthy foods to local communities. We believe that everyone deserves access to nutritious food, regardless of their location or socioeconomic status. Our mission is to make healthy eating a reality for all, while promoting sustainability and supporting local economies.\n\n### Services\n\nHealthym offers a wide range of healthy food products, including:\n\n* Fresh produce: seasonal fruits and vegetables sourced from local farmers\n* Whole grains: artisanal bread, pasta, and rice from small-scale producers\n* Plant-based protein: organic tofu, tempeh, and seitan from local suppliers\n* Specialty foods: artisanal cheeses, fermented foo

In [20]:
# Check number of documents in knowledge base

len(documents)

68

In [21]:
# Split the documents into smallers chunks to be used by a llm as context

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

Created a chunk of size 1931, which is longer than the specified 1000
Created a chunk of size 1105, which is longer than the specified 1000
Created a chunk of size 1029, which is longer than the specified 1000
Created a chunk of size 1005, which is longer than the specified 1000
Created a chunk of size 1303, which is longer than the specified 1000


In [ ]:
# Inspect number of generated chunks

len(chunks)

233

In [ ]:
# Che

doc_types = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Document types found: {', '.join(doc_types)}")

Document types found: company, products, recipes, suppliers


In [6]:
# Define text embedding model

embeddings = OpenAIEmbeddings()

# Could also use HuggingFaceEmbeddings if one is looking for a free alternative or if one has another specific model in mind
# from langchain.embeddings import HuggingFaceEmbeddings
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [28]:
# Check if a Chroma Datastore already exists.chunks
# If it exists, delete it to start over

if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()


In [ ]:
# Create Chroma vectorstore

vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

In [ ]:
# Get one sample vector and display its dimensions

collection = vectorstore._collection
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"The vectors have {dimensions:,} dimensions")

The vectors have 1,536 dimensions


In [8]:
# Create the chat instance to be used in the conversation chain
llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

# Define the memory to be used in the conversation chain
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# Define the retriever to be used in the conversation chain when retrieving relevant documents from the vector store.
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

# Create the conversation chain using the llm, memory, and retriever previously defined
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

In [65]:
# Test the application with sample queries

query = "Can you provide a short description of what Healthym does?"
result = conversation_chain.invoke({"question":query})

In [66]:
print(result['answer'])

Healthym is a pioneering company dedicated to providing high-quality, healthy foods to local communities. They offer a wide range of products, including fresh produce, whole grains, plant-based proteins, specialty foods, and meal kits. Healthym's mission is to make healthy eating accessible to everyone while promoting sustainability and supporting local economies.


In [89]:
result2 = conversation_chain.invoke({"question":"What Healthym sells?"})
print(result2['answer'])

Healthym offers a wide range of healthy food products, including:

- Fresh produce: seasonal fruits and vegetables sourced from local farmers
- Whole grains: artisanal bread, pasta, and rice from small-scale producers
- Plant-based protein: organic tofu, tempeh, and seitan from local suppliers
- Specialty foods: artisanal cheeses, fermented foods, and international cuisine
- Meal kits: pre-portioned ingredients and recipes for easy meal prep

Additionally, they provide meal kits catering to different dietary needs, including vegetarian, vegan, gluten-free, and keto-friendly options.


In [91]:
# Wrap the LangChain conversation chain in a function to be used with Gradio interface

def chat(message, history): # History is not used in this case (history is stored in LangChain), but it is required by Gradio CahtInterface
    result = conversation_chain.invoke({"question": message})
    return result["answer"]

In [ ]:
# Launch chatbot using Gradio

view = gr.ChatInterface(chat, 
                        type="messages", 
                        title="Healthym Chatbot",
                        description="Ask me anything about Healthym",
                        theme="soft")

view.launch()

### Final remarks
The chat agent work as intended and could be used by Healthym as part of customers' support.